In [4]:
import os
import math
from pathlib import Path
from typing import Tuple

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

# assume utils.py is in same folder and contains load_data, plot_history_tf, plot_heat_map
from utils import load_data, plot_history_tf, plot_heat_map

In [5]:
# -------------------------
# Configuration
# -------------------------
IMG_SIZE = 224  # image height/width in pixels (square)
IMAGE_DIR = Path("./ecg_images")
NPY_IMAGE_DIR = Path("./ecg_images_npy")
NPY_SIGNAL_DIR = Path("./ecg_signals_npy")
MODEL_SAVE_PATH = Path("./ecg_hybrid_model.h5")

BATCH_SIZE = 32
EPOCHS = 50
RANDOM_SEED = 42
NUM_CLASSES = 15

# Create directories
IMAGE_DIR.mkdir(parents=True, exist_ok=True)
NPY_IMAGE_DIR.mkdir(parents=True, exist_ok=True)
NPY_SIGNAL_DIR.mkdir(parents=True, exist_ok=True)